In [ ]:
!unzip -oq /kaggle/input/carvana-image-masking-challenge/train.zip
!unzip -oq /kaggle/input/carvana-image-masking-challenge/train_masks.zip
!unzip -oq /kaggle/input/carvana-image-masking-challenge/train_masks.csv.zip

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import pandas as pd

In [ ]:
def rle_decode(mask_rle, shape=(1280, 1918, 1)):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background
    '''
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)

    s = mask_rle.split()
    
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    
    ends = starts + lengths    
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
        
    img = img.reshape(shape)
    return img


In [ ]:
df = pd.read_csv('./train_masks.csv')
df.shape

In [ ]:
val_size = 0.2
val_split = int(val_size * df.shape[0])

train_df = df[:-val_split]
val_df = df[-val_split:]

train_df.shape, val_df.shape

In [ ]:
train_df.head()

In [ ]:
img_name, mask_rle = train_df.iloc[0]

img = cv2.imread(f'./train/{img_name}')
mask = rle_decode(mask_rle)

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(25, 25))
ax[0].imshow(img)
ax[1].imshow(mask)

plt.show()

In [ ]:
train_df.sample(1)

In [ ]:
def keras_generator(df, batch_size, preprocess_input=None):
    while True:
        X_batch = []
        y_batch = []
        
        for i in range(batch_size):
            img_name, mask_rle = df.sample(1).values[0]
            img = cv2.imread(f'./train/{img_name}')
            mask = rle_decode(mask_rle)
            
            img = cv2.resize(img, (256, 256))
            mask = cv2.resize(mask, (256, 256))
            
            X_batch += [img]
            y_batch += [mask]

        if preprocess_input:
            X_batch = preprocess_input(np.array(X_batch))
        else:
            X_batch = np.array(X_batch) / 255.0
            
        y_batch = np.array(y_batch, dtype='float')

        yield X_batch, y_batch

In [ ]:
for X, y in keras_generator(val_df, batch_size=32):
    print(X.shape, y.shape)
    break

## FCN
<img src='https://drive.google.com/uc?export=view&id=1j35v8z17TD6RgVTp2DoEokkCt44x71If' width=550>

In [ ]:
import keras
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, UpSampling2D, Conv2D, MaxPooling2D, Activation


base_model = VGG16(weights='imagenet', input_shape=(256, 256, 3), include_top=False)
base_out = base_model.output
base_out

In [ ]:
# up = UpSampling2D(32)(base_out)
up = UpSampling2D(32, interpolation='bilinear')(base_out)
up

In [ ]:
fcn = Model(inputs=base_model.input, outputs=up)
fcn.summary()

In [ ]:
pred = fcn.predict(X)
pred.shape

pred - карта с признаками (feature map), которая как-то характеризует картинку. 
Если посмотреть на любой канал у предсказания, то получим карту вероятностей.

Всё получилось пикселизировано, можно добавить линейную интерполяцию.

In [ ]:
plt.imshow(pred[0, :, :, 0])

Теперь feature map нужно сжать до того количества каналов, сколько у нас классов для масок, в нашем случае с машинами есть только один класс.

In [ ]:
conv = Conv2D(1, (1, 1), activation='sigmoid')(up)  # (1, 1) потому что не надо изменять изображение, хочется только сжать каналы
conv

In [ ]:
fcn = Model(inputs=base_model.input, outputs=conv)
fcn.summary()

In [ ]:
pred = fcn.predict(X)
pred.shape

In [ ]:
plt.imshow(pred[0]);

In [ ]:
checkpoint_best = keras.callbacks.ModelCheckpoint('fcn_best.h5',
                                                  monitor='val_loss',
                                                  verbose=0,
                                                  save_best_only=True,
                                                  save_weights_only=False,
                                                  mode='auto',
                                                  period=1)

checkpoint_last = keras.callbacks.ModelCheckpoint('fcn_last.h5',
                                                  monitor='val_loss',
                                                  verbose=0,
                                                  save_best_only=False,
                                                  save_weights_only=False,
                                                  mode='auto',
                                                  period=1)


callbacks = [checkpoint_best, checkpoint_last]



adam = keras.optimizers.Adam(lr=0.0001)

fcn.compile(adam, 'binary_crossentropy')

In [ ]:
batch_size = 16

fcn.fit(keras_generator(train_df, batch_size), steps_per_epoch=100,
        epochs=2, verbose=1,
        callbacks=callbacks,
        validation_data=keras_generator(val_df, batch_size), validation_steps=50)

In [ ]:
fcn.evaluate(keras_generator(val_df, batch_size), steps=25)

In [ ]:
pred = fcn.predict(X)
pred.shape

На выходе у нас будет не бинарная маска, а маска вероятностей.

In [ ]:
idx = 1
fig, axes = plt.subplots(1, 2, figsize=(15, 15))
axes[0].imshow(X[idx])
axes[1].imshow(pred[idx, ..., 0])

plt.show()

Чтобы перевести в бинарную маску, нужно сравнивать предсказанную вероятность с отсечкой.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 15))
axes[0].imshow(X[idx])
axes[1].imshow(pred[idx, ..., 0] > 0.5)

plt.show()

Большой недостаток рассмотренной архитектуры - слишком резко происходит увеличение изображения. Можно это смягчить за счет архитектуры SegNet.

## SegNet

<img src='https://drive.google.com/uc?export=view&id=1YRgQtgX90Y0YTuSRrDZfOr_qyEsVi6ax'>


In [ ]:
from keras.models import Model
from keras.layers import Input, Dense
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, UpSampling2D, Conv2D, MaxPooling2D


inp = Input(shape=(256, 256, 3))

conv_1_1 = Conv2D(32, (3, 3), padding='same', activation='relu')(inp)
conv_1_2 = Conv2D(32, (3, 3), padding='same', activation='relu')(conv_1_1)

pool_1 = MaxPooling2D(2)(conv_1_2)


conv_2_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(pool_1)
conv_2_2 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv_2_1)

pool_2 = MaxPooling2D(2)(conv_2_2)


conv_3_1 = Conv2D(128, (3, 3), padding='same', activation='relu')(pool_2)
conv_3_2 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv_3_1)

pool_3 = MaxPooling2D(2)(conv_3_2)


conv_4_1 = Conv2D(256, (3, 3), padding='same', activation='relu')(pool_3)
conv_4_2 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv_4_1)

pool_4 = MaxPooling2D(2)(conv_4_2)
pool_4

In [ ]:
up_1 = UpSampling2D(2, interpolation='bilinear')(pool_4)

conv_up_1_1 = Conv2D(256, (3, 3), padding='same', activation='relu')(up_1)
conv_up_1_2 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv_up_1_1)


up_2 = UpSampling2D(2, interpolation='bilinear')(conv_up_1_2)

conv_up_2_1 = Conv2D(128, (3, 3), padding='same', activation='relu')(up_2)
conv_up_2_2 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv_up_2_1)


up_3 = UpSampling2D(2, interpolation='bilinear')(conv_up_2_2)

conv_up_3_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(up_3)
conv_up_3_2 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv_up_3_1)


up_4 = UpSampling2D(2, interpolation='bilinear')(conv_up_3_2)

conv_up_4_1 = Conv2D(32, (3, 3), padding='same', activation='relu')(up_4)
conv_up_4_2 = Conv2D(1, (3, 3), padding='same', activation='sigmoid')(conv_up_4_1)
conv_up_4_2

In [ ]:
segnet = Model(inputs=inp, outputs=conv_up_4_2)
segnet.summary()

In [ ]:
checkpoint_best = keras.callbacks.ModelCheckpoint('segnet_best.h5',
                                                  monitor='val_loss',
                                                  verbose=0,
                                                  save_best_only=True,
                                                  save_weights_only=False,
                                                  mode='auto',
                                                  period=1)

checkpoint_last = keras.callbacks.ModelCheckpoint('segnet_last.h5',
                                                  monitor='val_loss',
                                                  verbose=0,
                                                  save_best_only=False,
                                                  save_weights_only=False,
                                                  mode='auto',
                                                  period=1)


callbacks = [checkpoint_best, checkpoint_last]

adam = keras.optimizers.Adam(lr=0.0001)

segnet.compile(adam, 'binary_crossentropy')

In [ ]:
batch_size = 16

segnet.fit(keras_generator(train_df, batch_size), steps_per_epoch=100,
           epochs=3, verbose=1,
           callbacks=callbacks,
           validation_data=keras_generator(val_df, batch_size), validation_steps=50)

In [ ]:
segnet.evaluate(keras_generator(val_df, batch_size), steps=25)

In [ ]:
pred = segnet.predict(X)
pred.shape

In [ ]:
idx = 0
fig, axes = plt.subplots(1, 2, figsize=(15, 15))
axes[0].imshow(X[idx])
axes[1].imshow(pred[idx, ..., 0] > 0.5)

plt.show()

## U-net
<img src='https://drive.google.com/uc?Export=view&id=1B_SOnL99Qwc-Yrka0kK7V5dTH3mJQZwN'>

In [ ]:
from keras.models import Model
from keras.layers import Input, Dense, Concatenate
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, UpSampling2D, Conv2D, MaxPooling2D

inp = Input(shape=(256, 256, 3))


# Downsampling
conv_1_1 = Conv2D(32, (3, 3), padding='same', activation='relu')(inp)
conv_1_2 = Conv2D(32, (3, 3), padding='same')(conv_1_1)

pool_1 = MaxPooling2D(2)(conv_1_2)


conv_2_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(pool_1)
conv_2_2 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv_2_1)

pool_2 = MaxPooling2D(2)(conv_2_2)


conv_3_1 = Conv2D(128, (3, 3), padding='same', activation='relu')(pool_2)
conv_3_2 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv_3_1)

pool_3 = MaxPooling2D(2)(conv_3_2)


# Bottleneck
conv_4_1 = Conv2D(256, (3, 3), padding='same', activation='relu')(pool_3)
conv_4_2 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv_4_1)

pool_4 = MaxPooling2D(2)(conv_4_2)
pool_4

In [ ]:
# Upsampling
up_1 = UpSampling2D(2, interpolation='bilinear')(pool_4)
conc_1 = Concatenate()([conv_4_2, up_1])

conv_up_1_1 = Conv2D(256, (3, 3), padding='same', activation='relu')(conc_1)
conv_up_1_2 = Conv2D(256, (3, 3), padding='same', activation='relu')(conv_up_1_1)


up_2 = UpSampling2D(2, interpolation='bilinear')(conv_up_1_2)
conc_2 = Concatenate()([conv_3_2, up_2])

conv_up_2_1 = Conv2D(128, (3, 3), padding='same', activation='relu')(conc_2)
conv_up_2_2 = Conv2D(128, (3, 3), padding='same', activation='relu')(conv_up_2_1)


up_3 = UpSampling2D(2, interpolation='bilinear')(conv_up_2_2)
conc_3 = Concatenate()([conv_2_2, up_3])

conv_up_3_1 = Conv2D(64, (3, 3), padding='same', activation='relu')(conc_3)
conv_up_3_2 = Conv2D(64, (3, 3), padding='same', activation='relu')(conv_up_3_1)


up_4 = UpSampling2D(2, interpolation='bilinear')(conv_up_3_2)
conc_4 = Concatenate()([conv_1_2, up_4])

conv_up_4_1 = Conv2D(32, (3, 3), padding='same', activation='relu')(conc_4)
conv_up_4_2 = Conv2D(1, (3, 3), padding='same', activation='sigmoid')(conv_up_4_1)
conv_up_4_2

In [ ]:
unet = Model(inputs=inp, outputs=conv_up_4_2)
unet.summary()

In [ ]:
checkpoint_best = keras.callbacks.ModelCheckpoint('unet_best.h5',
                                                  monitor='val_loss',
                                                  verbose=0,
                                                  save_best_only=True,
                                                  save_weights_only=False,
                                                  mode='auto',
                                                  period=1)

checkpoint_last = keras.callbacks.ModelCheckpoint('unet_last.h5',
                                                  monitor='val_loss',
                                                  verbose=0,
                                                  save_best_only=False,
                                                  save_weights_only=False,
                                                  mode='auto',
                                                  period=1)


callbacks = [checkpoint_best, checkpoint_last]

adam = keras.optimizers.Adam(lr=0.0001)

unet.compile(adam, 'binary_crossentropy')

In [ ]:
batch_size = 16

unet.fit(keras_generator(train_df, batch_size), steps_per_epoch=100,
         epochs=3, verbose=1,
         callbacks=callbacks,
         validation_data=keras_generator(val_df, batch_size), validation_steps=50)

In [ ]:
unet.evaluate(keras_generator(val_df, batch_size), steps=25)

In [ ]:
pred = unet.predict(X)
pred.shape

In [ ]:
idx = 0
fig, axes = plt.subplots(1, 2, figsize=(15, 15))
axes[0].imshow(X[idx])
axes[1].imshow(pred[idx, ..., 0] > 0.5)

plt.show()

## U-net + ResNet50

<img src='https://drive.google.com/uc?export=view&id=1mAxaE49NHBSJp3wXzu-z_4LYcoRGYY0t'>

In [ ]:
import keras
from keras.applications.resnet50 import ResNet50, preprocess_input
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, UpSampling2D, Conv2D, MaxPooling2D, Concatenate, Activation
from keras import backend as K


base_model = ResNet50(weights='imagenet', input_shape=(256, 256, 3), include_top=False)
 
base_out = base_model.output
base_out

In [ ]:
# len(base_model.layers)

In [ ]:
base_model.summary()

In [ ]:
conv1 = base_model.get_layer('conv1_relu').output  # (128, 128, 64)
conv2 = base_model.get_layer('conv2_block1_out').output  # (64, 64, 256)
conv3 = base_model.get_layer('conv3_block1_1_relu').output  # (32, 32, 128)
conv4 = base_model.get_layer('conv4_block2_2_relu').output  # (16, 16, 256)
conv5 = base_model.get_layer('conv5_block1_2_relu').output  # (8, 8, 512)


inp = base_model.get_layer('input_4').output

In [ ]:
conv5

In [ ]:
up1 = UpSampling2D(2, interpolation='bilinear')(conv5)
conc_1 = Concatenate()([up1, conv4])
print(conc_1)
conv_conc_1 = Conv2D(256, (3, 3), padding='same', activation='relu')(conc_1)

In [ ]:
up2 = UpSampling2D(2, interpolation='bilinear')(conv_conc_1)
conc_2 = Concatenate()([up2, conv3])
conv_conc_2 = Conv2D(128, (3, 3), padding='same', activation='relu')(conc_2)

up3 = UpSampling2D(2, interpolation='bilinear')(conv_conc_2)
conc_3 = Concatenate()([up3, conv2])
conv_conc_3 = Conv2D(64, (3, 3), padding='same', activation='relu')(conc_3)

up4 = UpSampling2D(2, interpolation='bilinear')(conv_conc_3)
conc_4 = Concatenate()([up4, conv1])
conv_conc_4 = Conv2D(32, (3, 3), padding='same', activation='relu')(conc_4)

up5 = UpSampling2D(2, interpolation='bilinear')(conv_conc_4)
conv_conc_5 = Conv2D(1, (3, 3), padding='same', activation='sigmoid')(up5)

In [ ]:
unet_50 = Model(inputs=base_model.input, outputs=conv_conc_5)
unet_50.summary()

In [ ]:
checkpoint_best = keras.callbacks.ModelCheckpoint('unet_50_best.h5',
                                                  monitor='val_loss',
                                                  verbose=0,
                                                  save_best_only=True,
                                                  save_weights_only=False,
                                                  mode='auto',
                                                  period=1)

checkpoint_last = keras.callbacks.ModelCheckpoint('unet_50_last.h5',
                                                  monitor='val_loss',
                                                  verbose=0,
                                                  save_best_only=False,
                                                  save_weights_only=False,
                                                  mode='auto',
                                                  period=1)


callbacks = [checkpoint_best, checkpoint_last]

adam = keras.optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

unet_50.compile(adam, 'binary_crossentropy')

In [ ]:
batch_size = 16

unet_50.fit(keras_generator(train_df, batch_size, preprocess_input),
            steps_per_epoch=100,
            epochs=3, verbose=1,
            callbacks=callbacks,
            validation_data=keras_generator(val_df, batch_size, preprocess_input),
            validation_steps=10)

In [ ]:
unet_50.evaluate(keras_generator(val_df, batch_size, preprocess_input), steps=25)

In [ ]:
for X, y in keras_generator(val_df, 16, preprocess_input):
    plt.imshow(X[0])
    print(X[0].min(), X[0].max())
    break

In [ ]:
pred = unet_50.predict(X)

In [ ]:
idx = 0
fig, axes = plt.subplots(1, 2, figsize=(15, 15))
axes[0].imshow(X[idx])
axes[1].imshow(pred[idx, ..., 0] > 0.5)

plt.show()

## Segmantational models

https://github.com/qubvel/segmentation_models

In [ ]:
!pip install segmentation-models

In [ ]:
import os
os.environ['SM_FRAMEWORK'] = 'tf.keras'
import segmentation_models as sm

BACKBONE = 'resnet34'
preprocess_input = sm.get_preprocessing(BACKBONE)

# define model
model = sm.Unet(BACKBONE, classes=1, encoder_weights='imagenet')

model.compile(
    keras.optimizers.Adam(lr=0.001),
    loss='binary_crossentropy',
)

# fit model
model.fit(keras_generator(train_df, batch_size, preprocess_input),
          steps_per_epoch=100,
          epochs=3, verbose=1,
          callbacks=callbacks,
          validation_data=keras_generator(val_df, batch_size, preprocess_input),
          validation_steps=10)

In [ ]:
model.evaluate(keras_generator(val_df, batch_size, preprocess_input), steps=25)

In [ ]:
for X, y in keras_generator(val_df, 16, preprocess_input):
    plt.imshow(X[0])
    print(X[0].min(), X[0].max())
    break

In [ ]:
pred = model.predict(X)

In [ ]:
idx = 0
fig, axes = plt.subplots(1, 2, figsize=(15, 15))
axes[0].imshow(X[idx])
axes[1].imshow(pred[idx, ..., 0] > 0.5)

plt.show()